# Using Sklearn

In [1]:
from sklearn import datasets
from sklearn.model_selection import train_test_split

In [2]:
iris = datasets.load_iris()
x_train ,x_test , y_train , y_test = train_test_split(iris.data , iris.target , random_state = 1) 

In [3]:
from sklearn.tree import DecisionTreeClassifier

In [4]:
clf = DecisionTreeClassifier()
clf.fit(x_train , y_train)

DecisionTreeClassifier()

# From Scratch

In [5]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import random
from pprint import pprint

# Load Data

In [199]:
or_data = np.loadtxt('xor.csv',delimiter = ',')
or_data

array([[0., 0., 0.],
       [0., 1., 1.],
       [1., 0., 1.],
       [1., 1., 1.]])

In [193]:
iris = datasets.load_iris()
df = pd.DataFrame(iris.data , columns = ['sl','sw','pl','pw'])
df['target'] = iris.target
df_latest = iris.target_names[df.target]
df.target = df_latest 

In [194]:
df.head()

,sl,sw,pl,pw,target
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


# Train-Test-Split

In [8]:
def train_test_split(df , test_size):
    indices = df.index.tolist()
    test_indices = random.sample(population=indices , k= test_size)
    test_df = df.loc[test_indices]
    train_df = df.drop(test_indices)
    return train_df , test_df

In [190]:
random.seed(0)
train_df , test_df = train_test_split(df , 20)

# Helper Function

In [10]:
data = train_df.values

In [11]:
def pure_node(data):
    target = data[:,-1]
    target = np.unique(target)
    if(len(target) == 1):
        return True
    else:
        return False

In [12]:
def classify_data(data):
    target = data[:,-1]
    unique_class , count = np.unique(target , return_counts = True)
    
    index = count.argmax()
    classificaion = unique_class[index]
    
    return classificaion
     

### Potential split

In [13]:
# train_df.head()


In [14]:
def potential_split(data):
    split = {}
    row, col = data.shape 
    for col_index in range(col -1):
        split[col_index] = []
        val = data[:,col_index]
        unique_val = np.unique(val)
        
        for index in range(len(unique_val)):
            if index!=0:
                curr_val = unique_val[index]
                prev_val = unique_val[index-1]
                split_val = round((curr_val + prev_val)/2 , 2)
                split[col_index].append(split_val)
    return split
    

In [15]:
def split_data(data , split_col , split_val):
    split_col_val = data[:,split_col]
    data_below = data[split_col_val <= split_val]
    data_above = data[split_col_val > split_val]
    
    return data_below , data_above

In [16]:
# split_col = 3
# split_val = 1.05
# below , above = split_data(data , split_col , split_val)

In [17]:
# plotting_df = pd.DataFrame(data , columns = df.columns)
# sns.lmplot(data=plotting_df, x = "pw" , y ="pl", fit_reg = False)
# plt.vlines(x=split_val ,ymin=1,ymax=7 )

### Entropy 

In [18]:
def calculate_entropy(data):
    target = data[:,-1]
    t , counts = np.unique(target , return_counts= True)
    prob = counts / (counts.sum())
    entropy = sum(prob * -np.log2(prob))
    return entropy

In [19]:
# calculate_entropy(above)

In [25]:
def cal_overall_entropy(data_below , data_above):
    n_data_points = len(data_above) + len(data_below)
    p_above = len(data_above) / n_data_points
    p_below = len(data_below) / n_data_points

    overall_entropy = (p_above * calculate_entropy(data_above)) + (p_below * calculate_entropy(data_below))
    return overall_entropy

In [21]:
# cal_overall_entropy(below ,above)

In [22]:
def find_best_split(data , split):
    overall_entropy = 10000
    for col_index in split:
        for val in split[col_index]:
            data_below , data_above = split_data(data , col_index , val)
            curr_overall_entropy = cal_overall_entropy(data_below , data_above)

            if curr_overall_entropy <= overall_entropy:
                overall_entropy = curr_overall_entropy
                best_split_col = col_index
                best_split_val = val

    return best_split_col,best_split_val

In [105]:
def cal_split_info(data_below , data_above):
    len_total = len(data_below) + len(data_above)
    p_below = len(data_below) / len_total
    p_above = len(data_above) / len_total
    
    split_info = (-p_below)* np.log2(p_below) + (-p_above)* np.log2(p_above)
    
    return split_info

In [97]:
def find_best_split_using_gain_ratio(data , split):
    info_gain = 0
    
    for col_index in split:
        for val in split[col_index]:
            data_below , data_above = split_data(data , col_index , val)
            curr_overall_entropy = cal_overall_entropy(data_below , data_above)
            root_entopy = calculate_entropy(data)
            split_info = cal_split_info(data_below , data_above)
            curr_info_gain = (root_entopy - curr_overall_entropy)/split_info
#             print(curr_info_gain)
            if info_gain <= curr_info_gain:
                info_gain = curr_info_gain
                best_split_col = col_index
                best_split_val = val
        
    return best_split_col,best_split_val , info_gain

# Decision Tree

In [241]:
def decision_tree_alogo(df,counter=0 , min_samples = 2 , max_depth =5):
    
    # data preparation
    if counter == 0:
        global col_header
        col_header = df.columns
        data = df.values
    else:
        data = df
    # base case
    if (pure_node(data)) or (len(data) < min_samples) or (counter == max_depth):
        classification = classify_data(data)
        return classification
    # recursive part
    else:
        counter +=1
        split = potential_split(data)
        split_col , split_val,gain_ratio = find_best_split_using_gain_ratio(data ,split)
        below , above = split_data(data,split_col , split_val)
        t , counts = np.unique(data[:,-1] , return_counts= True) 
        # subtree
        feature_name = col_header[split_col]
        quen = "{} <= {} , gain={} , samples={} ,value={} ".format( feature_name, split_val,
                                                                round(gain_ratio,7),len(data),counts)
        sub_tree = {quen :[]}
        
        #find answer
        yes_ans = decision_tree_alogo(below,counter,min_samples,max_depth)
        no_ans = decision_tree_alogo(above,counter,min_samples,max_depth)
        if yes_ans == no_ans:
            sub_tree = yes_ans
        else:
            sub_tree[quen].append(yes_ans)
            sub_tree[quen].append(no_ans)
        
        return sub_tree

In [242]:
tree =decision_tree_alogo(train_df,max_depth = 5)
print(tree)

{'pw <= 0.8 , gain=1.0 , samples=130 ,value=[46 42 42] ': ['setosa', {'pw <= 1.65 , gain=0.7327835 , samples=84 ,value=[42 42] ': [{'pl <= 4.95 , gain=0.6492629 , samples=44 ,value=[41  3] ': ['versicolor', {'pw <= 1.55 , gain=1.0 , samples=4 ,value=[1 3] ': ['virginica', 'versicolor']}]}, {'pl <= 4.85 , gain=0.1866395 , samples=40 ,value=[ 1 39] ': [{'sw <= 3.1 , gain=1.0 , samples=4 ,value=[1 3] ': ['virginica', 'versicolor']}, 'virginica']}]}]}


# Coding Ninja

In [179]:
def decision_tree_steps(df,counter=0 , min_samples = 2 , max_depth =5):
    
    # data preparation
    if counter == 0:
        global col_header
        col_header = df.columns
        data = df.values
    else:
        data = df
    # base case
    
    if (pure_node(data)) or (len(data) < min_samples) or (counter == max_depth):
        if(counter == max_depth):
            return
        print('Level ',counter)
        iris_name,iris_count = (np.unique(data[:,-1], return_counts= True))        
        for i in range(len(iris_name)):
            print('Count of ',iris_name[i], ' = ' , iris_count[i])
        
        print('Current Entropy is =' ,calculate_entropy(data))
        print('Reached leaf Node')
        print()
        
    # recursive part
    
    else:
        print('Level ',counter)
        iris_name,iris_count = (np.unique(data[:,-1], return_counts= True))        
        for i in range(len(iris_name)):
            print('Count of ',iris_name[i], ' = ' , iris_count[i])
        
        print('Current Entropy is =' ,calculate_entropy(data))
        
        counter +=1
        split = potential_split(data)
        split_col , split_val , gain = find_best_split_using_gain_ratio(data , split)        
        below , above = split_data(data,split_col , split_val)
        # subtree
        feature_name = col_header[split_col]
        print('Splitting on feature' ,feature_name,'<=',split_val,'with gain ratio',gain )
        print()
        

        decision_tree_steps(below,counter,min_samples,max_depth)
        decision_tree_steps(above,counter,min_samples,max_depth)
        

In [206]:
decision_tree_steps(or_df,max_depth = 4)

Level  0
Count of  0.0  =  1
Count of  1.0  =  3
Current Entropy is = 0.8112781244591328
Splitting on feature x2 <= 0.5 with gain ratio 0.31127812445913283

Level  1
Count of  0.0  =  1
Count of  1.0  =  1
Current Entropy is = 1.0
Splitting on feature x1 <= 0.5 with gain ratio 1.0

Level  2
Count of  0.0  =  1
Current Entropy is = 0.0
Reached leaf Node

Level  2
Count of  1.0  =  1
Current Entropy is = 0.0
Reached leaf Node

Level  1
Count of  1.0  =  2
Current Entropy is = 0.0
Reached leaf Node



In [203]:
or_df = pd.DataFrame(or_data , columns = ['x1','x2','y'])

In [205]:
or_df

,x1,x2,y
0,0.0,0.0,0.0
1,0.0,1.0,1.0
2,1.0,0.0,1.0
3,1.0,1.0,1.0


In [230]:
print("{} <= {}\nSamples:{} ".format( 5,6 ,5+6))


5 <= 6
Samples:11 


In [216]:
print("ll")

l
l


In [240]:
list_print = []
for i in tree:
    list_print.append(i)
    print(i)
while(len(list_print)!=0):
    

In [233]:
tree

{'pw <= 0.8\ngain=1.0\nsamples=130\nvalue=[46 42 42] ': ['setosa',
  {'pw <= 1.65\ngain=0.7327835\nsamples=84\nvalue=[42 42] ': [{'pl <= 4.95\ngain=0.6492629\nsamples=44\nvalue=[41  3] ': ['versicolor',
      {'pw <= 1.55\ngain=1.0\nsamples=4\nvalue=[1 3] ': ['virginica',
        'versicolor']}]},
    {'pl <= 4.85\ngain=0.1866395\nsamples=40\nvalue=[ 1 39] ': [{'sw <= 3.1\ngain=1.0\nsamples=4\nvalue=[1 3] ': ['virginica',
        'versicolor']},
      'virginica']}]}]}